In [198]:
import numpy as np
import pandas as pd
import datatable as dt
from itertools import groupby
import json
from tqdm import tqdm

In [207]:
df = dt.fread("../../../data/events_sorted.csv")

Reading file: [#############################################              ]  77%

In [208]:
df = df[:100000,:].topandas()

In [209]:
df = df.sort_values(['user_id', 'session_id', 'step'])

In [210]:
df["clickout_step_rev"][df["action_type"] != "clickout item"] = 0

/home/pawel/.pyenv/versions/miniconda3-latest/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [211]:
df.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,src,is_test,is_val,fake_impressions,fake_prices,clickout_step_rev,clickout_step,clickout_max_step
43228,002J73UJ7Z1T,f721d12954e48,1541032764,1,interaction item deals,3864582,BR,"Aracaju, Brazil",mobile,,,,train,0,0,1776335|1827211|1827619|1827699|1828421|194701...,73|60|59|52|81|68|72|90|72|72|47|54|79|53|72|7...,0,1,3
45089,002J73UJ7Z1T,f721d12954e48,1541032842,2,interaction item deals,6623894,BR,"Aracaju, Brazil",mobile,,,,train,0,0,1776335|1827211|1827619|1827699|1828421|194701...,73|60|59|52|81|68|72|90|72|72|47|54|79|53|72|7...,0,2,3
45301,002J73UJ7Z1T,f721d12954e48,1541032852,3,interaction item image,6623894,BR,"Aracaju, Brazil",mobile,,,,train,0,0,1776335|1827211|1827619|1827699|1828421|194701...,73|60|59|52|81|68|72|90|72|72|47|54|79|53|72|7...,0,1,47
45302,002J73UJ7Z1T,f721d12954e48,1541032852,4,interaction item image,6623894,BR,"Aracaju, Brazil",mobile,,,,train,0,0,1776335|1827211|1827619|1827699|1828421|194701...,73|60|59|52|81|68|72|90|72|72|47|54|79|53|72|7...,0,2,47
45559,002J73UJ7Z1T,f721d12954e48,1541032862,5,interaction item image,6623894,BR,"Aracaju, Brazil",mobile,,,,train,0,0,1776335|1827211|1827619|1827699|1828421|194701...,73|60|59|52|81|68|72|90|72|72|47|54|79|53|72|7...,0,3,47


In [121]:
df['timestamp_next'] = df.groupby(["user_id" ,"session_id"])["timestamp"].shift(-1).fillna(0).astype(np.int)
df['action_type_prv'] = df.groupby(["user_id" ,"session_id"])["action_type"].shift(1)
df['session_id_prv'] = df.groupby(["user_id" ,"session_id"])["session_id"].shift(1)

In [81]:
df['break_point'] = (df["reference"]!=df["reference"]) | (df["action_type"]!=df["action_type_prv"]) | (df["session_id"]!=df["session_id_prv"])
df['break_point_cumsum'] = df.groupby(['user_id', 'session_id'])['break_point'].cumsum()

In [206]:
def get_index_clicked(row):
    if not row["reference"].isnumeric():
        return 'UNK'
    item_id = int(row["reference"])
    try:
        impressions = list(map(int,row["fake_impressions"].split("|")))
    except ValueError:
        return 'UNK'
    if item_id in impressions:
        return str(impressions.index(item_id))
    else:
        return 'UNK'

def get_price_clicked(row):
    if not row["reference"].isnumeric():
        return 0
    item_id = int(row["reference"])
    try:
        impressions = list(map(int,row["fake_impressions"].split("|")))
        prices = list(map(int,row["fake_prices"].split("|")))
    except ValueError:
        return 0
    if item_id in impressions:
        return prices[impressions.index(item_id)]
    else:
        return 0
    
df['index_clicked'] = df.apply(get_index_clicked, axis=1)
df['price_clicked'] = df.apply(get_price_clicked, axis=1)

AttributeError: 'Frame' object has no attribute 'apply'

In [126]:
for _, df_session in df.iloc[:10000].groupby("session_id") :
    session_obs = df_session.to_dict(orient='records')
    break

In [127]:
df[['session_id', 'action_type', 'reference', 'break_point', 'break_point_cumsum', 'timestamp', 'timestamp_next']].head(100)

,session_id,action_type,reference,break_point,break_point_cumsum,timestamp,timestamp_next
43228,f721d12954e48,interaction item deals,3864582,True,1.0,1541032764,1541032842
45089,f721d12954e48,interaction item deals,6623894,False,1.0,1541032842,1541032852
45301,f721d12954e48,interaction item image,6623894,True,2.0,1541032852,1541032852
45302,f721d12954e48,interaction item image,6623894,False,2.0,1541032852,1541032862
45559,f721d12954e48,interaction item image,6623894,False,2.0,1541032862,1541032862
45560,f721d12954e48,interaction item image,6623894,False,2.0,1541032862,1541032862
45561,f721d12954e48,interaction item image,6623894,False,2.0,1541032862,1541032862
45562,f721d12954e48,interaction item image,6623894,False,2.0,1541032862,1541032862
45563,f721d12954e48,interaction item image,6623894,False,2.0,1541032862,1541032862
45564,f721d12954e48,interaction item image,6623894,False,2.0,1541032862,1541032862


In [192]:
df_no_dup = df.groupby(['src', 'is_val', 'is_test', 'user_id', 'session_id', 'platform', 'device', 'city', 'action_type', 'reference', 'break_point_cumsum', 'fake_prices', 'index_clicked', 'clickout_step_rev']).agg({
    'timestamp': ['min', 'max', 'count'],
    'timestamp_next': ['max', 'min'],
    'step': ['min']
}).reset_index()
df_no_dup.columns = ['_'.join(col).strip('_') for col in df_no_dup.columns.values]
df_no_dup.sort_values(['user_id', 'session_id', 'step_min']).head(25)

,src,is_val,is_test,user_id,session_id,platform,device,city,action_type,reference,break_point_cumsum,fake_prices,index_clicked,clickout_step_rev,timestamp_min,timestamp_max,timestamp_count,timestamp_next_max,timestamp_next_min,step_min
5,train,0,0,002J73UJ7Z1T,f721d12954e48,BR,mobile,"Aracaju, Brazil",interaction item deals,3864582,1.0,73|60|59|52|81|68|72|90|72|72|47|54|79|53|72|7...,18,0,1541032764,1541032764,1,1541032842,1541032842,1
6,train,0,0,002J73UJ7Z1T,f721d12954e48,BR,mobile,"Aracaju, Brazil",interaction item deals,6623894,1.0,73|60|59|52|81|68|72|90|72|72|47|54|79|53|72|7...,23,0,1541032842,1541032842,1,1541032852,1541032852,2
8,train,0,0,002J73UJ7Z1T,f721d12954e48,BR,mobile,"Aracaju, Brazil",interaction item image,6623894,2.0,73|60|59|52|81|68|72|90|72|72|47|54|79|53|72|7...,23,0,1541032852,1541032925,45,1541032934,1541032852,3
7,train,0,0,002J73UJ7Z1T,f721d12954e48,BR,mobile,"Aracaju, Brazil",interaction item deals,6623894,3.0,73|60|59|52|81|68|72|90|72|72|47|54|79|53|72|7...,23,0,1541032934,1541032934,1,1541032986,1541032986,48
9,train,0,0,002J73UJ7Z1T,f721d12954e48,BR,mobile,"Aracaju, Brazil",interaction item image,6623894,4.0,73|60|59|52|81|68|72|90|72|72|47|54|79|53|72|7...,23,0,1541032986,1541032986,2,1541032990,1541032986,49
3,train,0,0,002J73UJ7Z1T,f721d12954e48,BR,mobile,"Aracaju, Brazil",clickout item,6623894,5.0,73|60|59|52|81|68|72|90|72|72|47|54|79|53|72|7...,23,5,1541032990,1541032990,1,1541033032,1541033032,51
2,train,0,0,002J73UJ7Z1T,f721d12954e48,BR,mobile,"Aracaju, Brazil",clickout item,6623894,5.0,73|60|59|52|81|68|72|90|72|72|47|54|79|53|72|7...,23,4,1541033032,1541033032,1,1541033055,1541033055,52
1,train,0,0,002J73UJ7Z1T,f721d12954e48,BR,mobile,"Aracaju, Brazil",clickout item,6623894,5.0,73|60|59|52|81|68|72|90|72|72|47|54|79|53|72|7...,23,3,1541033055,1541033055,1,1541033273,1541033273,53
0,train,0,0,002J73UJ7Z1T,f721d12954e48,BR,mobile,"Aracaju, Brazil",clickout item,6623894,5.0,73|60|59|52|81|68|72|90|72|72|47|54|79|53|72|7...,23,2,1541033273,1541033273,1,1541033419,1541033419,54
4,train,0,0,002J73UJ7Z1T,f721d12954e48,BR,mobile,"Aracaju, Brazil",clickout item,6623894,5.0,73|60|59|52|81|68|73|90|72|72|47|54|79|53|72|7...,23,1,1541033419,1541033419,1,0,0,55


In [204]:
def convert_session_df(df_session):
    session = df_session.to_dict(orient='records')
    session = sorted(session, key=lambda x: x['step_min'])
    if session[-1]['action_type'] != 'clickout item':
        return None
    n = len(session)-1
    sequences = {
        'action type cat': ['NA']*n,
        'clickout item cat':['NA']*n,
        'interaction item info cat':['NA']*n,
        'interaction item rating cat':['NA']*n,
        'interaction item deals cat':['NA']*n,
        'interaction item image cat':['NA']*n,
        'search for item num':['NA']*n,
        
        'clickout item num':[-1]*n,
        'interaction item info num':[-1]*n,
        'interaction item rating num':[-1]*n,
        'interaction item deals num':[-1]*n,
        'interaction item image num':[-1]*n,

        'filter selection cat':['NA']*n,
        'change of sort order cat':['NA']*n,
        'search for destination cat':['NA']*n,
        'search for item cat':['NA']*n,
        'search for poi cat':['NA']*n,

        'count action': [1]*n,
        'time spent': [0]*n,
        'timestamp diff': [0]*n
    }
    for i, obs in enumerate(session[:-1]):
        sequences['action type cat'][i] = obs['action_type']
        sequences['time spent'][i] = obs['timestamp_max'] - obs['timestamp_min']
        sequences['timestamp diff'][i] = max(0, obs['timestamp_next_min'] - obs['timestamp_max'])
        
        sequences['count action'][i] = obs['timestamp_count']
        if obs['index_clicked'] != 'UNK':
            sequences[obs['action_type'] + " cat"][i] = obs['index_clicked']
            sequences[obs['action_type'] + " num"][i] = int(obs['index_clicked'])
        else:
            if not obs['reference'].isnumeric():
                sequences[obs['action_type'] + " cat"][i] = obs['reference']

    session_info = {
        "src": session[0]["src"],
        "is_val": session[0]["is_val"],
        "is_test": session[0]["is_test"],
        "user_id": session[0]["user_id"],
        "session_id": session[0]["session_id"],
        "platform": session[0]["platform"],
        "city": session[0]["city"],
        "device": session[0]["device"],
        "index_clicked": session[-1]['index_clicked'],
        "final_prices": list(map(int, session[-1]["fake_prices"].split("|"))),
        "sequences": sequences
    }
    return session_info

In [ ]:
data = []
for _, df_session in tqdm(df_no_dup.groupby(["user_id", "session_id"])):
    session_info = convert_session_df(df_session)
    if session_info:
        data.append(session_info)